In [1]:
import numpy as np
import pandas as pd
from sklearn import cluster
import atd2025
import gc
import itertools as it

### Setup

Read dataset, note that to run the algorithms on an entire day of the dataset requires about 500GB of memory, as we give the most time efficient version of the distance matrix calculation algorithm as possible. Requires atd2025 package, available from https://gitlab.com/algorithms-for-threat-detection/2025/atd2025.

In [2]:
# Read dataset in
data = pd.read_csv("https://www.maserv.work/ATD/model2/ucf_atd_model/datasets/dataset1_truth.csv")
data["time"] = pd.to_datetime(data["time"])

Verify dataset is loaded:

In [3]:
data.head()

,point_id,point_id_original,track_id,time,lat,lon,speed,course
0,0,603,338214987,2024-01-01,28.033870,-96.974543,6.5,56.1
1,1,1474,366970770,2024-01-01,30.010361,-90.726282,2.9,270.3
2,2,3120,368013670,2024-01-01,30.708593,-88.039977,0.0,156.5
3,3,3276,538002778,2024-01-01,30.006844,-90.472937,12.9,158.0
4,4,1219,368014410,2024-01-01,28.174239,-82.818806,2.5,74.8


In [4]:
data.tail()

,point_id,point_id_original,track_id,time,lat,lon,speed,course
102856,102856,7742624,367782270,2024-01-01 23:59:59,30.224624,-93.263032,0.0,329.1
102857,102857,7734539,368092860,2024-01-01 23:59:59,27.819706,-97.458637,6.4,259.3
102858,102858,7741982,367109000,2024-01-01 23:59:59,28.892412,-94.211353,14.2,104.2
102859,102859,7735755,366996140,2024-01-01 23:59:59,29.763715,-95.095197,6.5,83.9
102860,102860,7738673,367003880,2024-01-01 23:59:59,30.766083,-88.051385,6.4,115.2


### Algorithm

Algorithm to create distance matrix between all pairs of points in dataset, according to cbtr rules

In [ ]:
def bidirectional_distance(x1, y1, t1, speed1, course1, time_window):
    # Setup proper broadcasting rules
    x2 = x1[:, None]
    y2 = y1[:, None]
    t2 = t1[:, None]
    speed2 = speed1[:, None]
    course2 = course1[:, None]

    time_window = time_window.astype("float32")

    x1 = x1[None, :]
    y1 = y1[None, :]
    t1 = t1[None, :]
    speed1 = speed1[None, :]
    course1 = course1[None, :]

    knots_to_mps = 0.514444
    
    # Forward prediction
    dt = (t2 - t1).astype("timedelta64[s]").astype("float32")
    pred_x = x1 + speed1 * np.sin(course1) * dt * knots_to_mps
    pred_y = y1 + speed1 * np.cos(course1) * dt * knots_to_mps
    forward_dist = np.square(x2 - pred_x) + np.square((y2 - pred_y))

    # Save some memory
    del pred_x
    del pred_y

    # Backward prediction
    back_x = x2 - speed2 * np.sin(course2) * dt * knots_to_mps
    back_y = y2 - speed2 * np.cos(course2) * dt * knots_to_mps
    backward_dist = np.square(x1 - back_x) + np.square(y1 - back_y)

    # Save some memory
    del back_x
    del back_y
    
    dist = 0.5 * (forward_dist + backward_dist)

    # Ensure that we only look within the next time_window if speed is not 0. 
    toInf = (dt <= 0) | ((dt > time_window) & (speed1 != 0)) | (dt < 2)
    
    dist[toInf] = np.inf
    
    return dist

Preprocess data for correct format for distance matrix creator, uses spherical location projection here, can also use UTM/whatever 2d coordinate system is preferred.

In [6]:
rad_earth = 6371000

# Approximately convert lon/lat to x/y in meters
def lonlat_to_xy(lon, lat):
    return lon * (np.pi / 180) * rad_earth * np.cos(lat * (np.pi / 180)), lat * rad_earth * (np.pi / 180)

# Approximately convert back to lon/lat
def xy_to_lonlat(x, y):
    lat = y * (180 / (np.pi * rad_earth))
    return x * (180 / (np.pi * rad_earth * np.cos(lat * (np.pi / 180)))), lat

# Read in coordinates
lon = data["lon"].to_numpy().astype("float32")
lat = data["lat"].to_numpy().astype("float32")

# Convert to meters
x, y = lonlat_to_xy(lon, lat)

x = x.astype("float32")
y = y.astype("float32")

# Read in rest of data
t = data["time"].to_numpy()
speed = data["speed"].to_numpy().astype("float32")
course = data["course"].to_numpy().astype("float32") * np.pi / 180

gc.collect()

40

Actually run the algorithm, ds_size controls how much data to run the algorithm on, set to 30000 to run on machines with 32GB of memory for hyperparameter tuning purposes.

In [ ]:
print("Start")

ds_size = 30000
ground = data[:ds_size]
data[:ds_size].to_csv("ground.csv")

# Initialize hyperparameters
time_window = 2100 # Should be somewhere around 1800
windowLen = np.array([time_window], dtype="timedelta64[s]")

# Create distance matrix, get top 10 values and their indices.
dist_matrix = bidirectional_distance(x[:ds_size], y[:ds_size], t[:ds_size], speed[:ds_size], course[:ds_size], windowLen[:ds_size])

Start


In [10]:
data["track_id"][:ds_size].unique()

array([338214987, 366970770, 368013670, ..., 367601420, 636017988,
       367343230], shape=(4627,))

In [11]:
(data["track_id"][:ds_size].value_counts() > 1).sum()

np.int64(2850)

In [12]:
dist_matrix[dist_matrix == np.inf] = 1e30

DBSCAN Posit accuracy

In [ ]:
dbscan = cluster.DBSCAN(eps=40000, metric="precomputed")
out = dbscan.fit_predict(dist_matrix.T)
out[out == -1] = np.arange(np.max(out) + 1, np.max(out) + 1 + np.sum(out == -1))
toGrade = pd.DataFrame({"point_id": data[:ds_size]["point_id"], "track_id": out})
toGrade.to_csv("clustering_output/grademe_dbscan.csv")

In [5]:
atd2025.accuracy.evaluate_predictions("clustering_output/grademe_dbscan.csv", "ground.csv")

0.11006601141346088

Average Linkage Posit accuracy

In [ ]:
average = cluster.AgglomerativeClustering(distance_threshold=80000000, metric="precomputed", linkage="average", n_clusters=None)
out = average.fit_predict(dist_matrix.T)
out[out == -1] = np.arange(np.max(out) + 1, np.max(out) + 1 + np.sum(out == -1))
toGrade = pd.DataFrame({"point_id": data[:ds_size]["point_id"], "track_id": out})
toGrade.to_csv("clustering_output/grademe_avg.csv")

In [6]:
atd2025.accuracy.evaluate_predictions("clustering_output/grademe_avg.csv", "ground.csv")

0.1358435169792244

Complete Linkage Posit accuracy

In [ ]:
average = cluster.AgglomerativeClustering(distance_threshold=100000000, metric="precomputed", linkage="complete", n_clusters=None)
out = average.fit_predict(dist_matrix.T)
out[out == -1] = np.arange(np.max(out) + 1, np.max(out) + 1 + np.sum(out == -1))
toGrade = pd.DataFrame({"point_id": data[:ds_size]["point_id"], "track_id": out})
toGrade.to_csv("grademe.csv")
atd2025.accuracy.evaluate_predictions("clustering_output/grademe_complete.csv", "ground.csv")

In [7]:
atd2025.accuracy.evaluate_predictions("clustering_output/grademe_complete.csv", "ground.csv")

0.14289186377733057

Single Linkage Posit accuracy

In [ ]:
average = cluster.AgglomerativeClustering(distance_threshold=100000, metric="precomputed", linkage="single", n_clusters=None)
out = average.fit_predict(dist_matrix.T)
out[out == -1] = np.arange(np.max(out) + 1, np.max(out) + 1 + np.sum(out == -1))
toGrade = pd.DataFrame({"point_id": data[:ds_size]["point_id"], "track_id": out})
toGrade.to_csv("grademe.csv")
atd2025.accuracy.evaluate_predictions("clustering_output/grademe_single.csv", "ground.csv")

In [8]:
atd2025.accuracy.evaluate_predictions("clustering_output/grademe_single.csv", "ground.csv")

0.1877630977727224

CBTR Posit accuracy

In [11]:
time_window = 2100
windowLen = np.array([time_window], dtype="timedelta64[s]")

closest = np.load("cbtr_closest/closest_ds1.npy")
closest_dists = np.load("cbtr_closest/closest_dists_ds1.npy")

# Ensure a consistent number is used to indicate inf distance
closest[closest_dists == np.inf] = closest.shape[1] - 1 

# Technically points that aren't colliders could be parallelized to some extent
collisions = (pd.Series(closest[0]).value_counts() > 1)

clusters = []
clusterNum = 0
clusterNums = [0] * closest.shape[1]
colliders = collisions[collisions].keys()
for i in range(closest.shape[1]):
    elems = closest[:, i]
    dists = closest_dists[:, i]
    if elems[0] == closest.shape[1] - 1:
        if not clusterNums[i]:
            clusterNums[i] = clusterNum
            clusters.append([i])
            clusterNum += 1
    else:
        if not clusterNums[i]:
            clusterNums[i] = clusterNum
            clustered = False
            for j, elem in enumerate(elems):
                if not clusterNums[elem]:
                    dist = dists[j]
                    collisionDists = closest_dists[:, i:][closest[:, i:] == elem]
                    if dist <= np.min(collisionDists):
                        clustered = True
                        clusterNums[elem] = clusterNum
                        clusters.append([i, elem])
                        break
            if not clustered:
                clusters.append([i])
            clusterNum += 1
        else:
            for j, elem in enumerate(elems):
                if not clusterNums[elem]:
                    dist = dists[j]
                    collisionDists = closest_dists[:, i:][closest[:, i:] == elem]
                    if dist <= np.min(collisionDists):
                        clusterNums[elem] = clusterNums[i]
                        clusters[clusterNums[i]].append(elem)
                        break
    if i % 1000 == 0:
        print(i)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000


In [12]:
zeroClusterIdx = [i for i, x in enumerate(clusters) if clusters[0][-1] in x][-1]
clusters[0] = clusters[0][:-1] + clusters[zeroClusterIdx]
clusters.remove(clusters[zeroClusterIdx])

point_ids = np.array(list(it.chain(*clusters)))
track_ids = np.array(list(it.chain(*[[i for z in x] for i, x in enumerate(clusters)])))
output = pd.DataFrame(np.array([point_ids, track_ids]).T, columns=["point_id", "track_id"])
output.to_csv("clustering_output/grademe_cbtr.csv", index=False)

In [13]:
atd2025.accuracy.evaluate_predictions("clustering_output/grademe_cbtr.csv", "https://www.maserv.work/ATD/model2/ucf_atd_model/datasets/dataset1_truth.csv")

0.38249190655350424